## 用RNN做文本生成
举个小小的例子，来看看LSTM是怎么玩的
我们这里用温斯顿丘吉尔的人物传记作为我们的学习语料。
第一步，一样，先导入各种库

In [1]:
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

Using TensorFlow backend.


接下来，我们把文本读入

In [4]:
raw_text = open('./input/Winston_Churchil.txt').read()
raw_text = raw_text.lower()

既然我们是以每个字母为层级，字母总共才26个，所以我们可以很方便的用One-Hot来编码出所有的字母（当然，可能还有些标点符号和其他noise）

In [5]:
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))

我们看到，全部的chars：

In [6]:
chars

['\n',
 ' ',
 '!',
 '#',
 '$',
 '%',
 '(',
 ')',
 '*',
 ',',
 '-',
 '.',
 '/',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 ';',
 '?',
 '@',
 '[',
 ']',
 '_',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '‘',
 '’',
 '“',
 '”',
 '\ufeff']

一共有：

In [7]:
len(chars)

61

同时，我们的原文本一共有:

In [8]:
len(raw_text)

276830

我们这里简单的文本预测就是，给了前置的字母以后，下一个字母是谁？

比如，Winsto, 给出 n Britai 给出 n

## 构造训练测试集

我们需要把我们的raw text变成可以用来训练的x,y:

x 是前置字母们 y 是后一个字母

In [10]:
seq_length = 100
x = []
y = []
for i in range(0, len(raw_text) - seq_length):
    given = raw_text[i:i + seq_length]
    predict = raw_text[i + seq_length]
    x.append([char_to_int[char] for char in given])
    y.append(char_to_int[predict])

我们可以看看我们做好的数据集的长相：

In [11]:
print(x[:3])
print(y[:3])

[[60, 45, 47, 44, 39, 34, 32, 49, 1, 36, 50, 49, 34, 43, 31, 34, 47, 36, 57, 48, 1, 47, 34, 30, 41, 1, 48, 44, 41, 33, 38, 34, 47, 48, 1, 44, 35, 1, 35, 44, 47, 49, 50, 43, 34, 9, 1, 31, 54, 1, 47, 38, 32, 37, 30, 47, 33, 1, 37, 30, 47, 33, 38, 43, 36, 1, 33, 30, 51, 38, 48, 0, 0, 49, 37, 38, 48, 1, 34, 31, 44, 44, 40, 1, 38, 48, 1, 35, 44, 47, 1, 49, 37, 34, 1, 50, 48, 34, 1, 44], [45, 47, 44, 39, 34, 32, 49, 1, 36, 50, 49, 34, 43, 31, 34, 47, 36, 57, 48, 1, 47, 34, 30, 41, 1, 48, 44, 41, 33, 38, 34, 47, 48, 1, 44, 35, 1, 35, 44, 47, 49, 50, 43, 34, 9, 1, 31, 54, 1, 47, 38, 32, 37, 30, 47, 33, 1, 37, 30, 47, 33, 38, 43, 36, 1, 33, 30, 51, 38, 48, 0, 0, 49, 37, 38, 48, 1, 34, 31, 44, 44, 40, 1, 38, 48, 1, 35, 44, 47, 1, 49, 37, 34, 1, 50, 48, 34, 1, 44, 35], [47, 44, 39, 34, 32, 49, 1, 36, 50, 49, 34, 43, 31, 34, 47, 36, 57, 48, 1, 47, 34, 30, 41, 1, 48, 44, 41, 33, 38, 34, 47, 48, 1, 44, 35, 1, 35, 44, 47, 49, 50, 43, 34, 9, 1, 31, 54, 1, 47, 38, 32, 37, 30, 47, 33, 1, 37, 30, 47, 33,

此刻，楼上这些表达方式，类似就是一个词袋，或者说 index。  
接下来我们做两件事：  
第一、我们已经有了一个input的数字表达（index），我们要把它变成LSTM需要的数组格式： [样本数，时间步伐，特征]  
第二、对于output，我们在Word2Vec里学过，用one-hot做output的预测可以给我们更好的效果，相对于直接预测一个准确的y数值的话。

In [12]:
n_patterns = len(x)
n_vocab = len(chars)

# 把x变成LSTM需要的样子
x = numpy.reshape(x, (n_patterns, seq_length, 1))
# 简单normal到0-1之间
x = x / float(n_vocab)
# output变成one-hot
y = np_utils.to_categorical(y)

print(x[11])
print(y[11])

[[0.80327869]
 [0.55737705]
 [0.70491803]
 [0.50819672]
 [0.55737705]
 [0.7704918 ]
 [0.59016393]
 [0.93442623]
 [0.78688525]
 [0.01639344]
 [0.7704918 ]
 [0.55737705]
 [0.49180328]
 [0.67213115]
 [0.01639344]
 [0.78688525]
 [0.72131148]
 [0.67213115]
 [0.54098361]
 [0.62295082]
 [0.55737705]
 [0.7704918 ]
 [0.78688525]
 [0.01639344]
 [0.72131148]
 [0.57377049]
 [0.01639344]
 [0.57377049]
 [0.72131148]
 [0.7704918 ]
 [0.80327869]
 [0.81967213]
 [0.70491803]
 [0.55737705]
 [0.14754098]
 [0.01639344]
 [0.50819672]
 [0.8852459 ]
 [0.01639344]
 [0.7704918 ]
 [0.62295082]
 [0.52459016]
 [0.60655738]
 [0.49180328]
 [0.7704918 ]
 [0.54098361]
 [0.01639344]
 [0.60655738]
 [0.49180328]
 [0.7704918 ]
 [0.54098361]
 [0.62295082]
 [0.70491803]
 [0.59016393]
 [0.01639344]
 [0.54098361]
 [0.49180328]
 [0.83606557]
 [0.62295082]
 [0.78688525]
 [0.        ]
 [0.        ]
 [0.80327869]
 [0.60655738]
 [0.62295082]
 [0.78688525]
 [0.01639344]
 [0.55737705]
 [0.50819672]
 [0.72131148]
 [0.72131148]
 [0.65

## 模型建造
LSTM模型构建

In [13]:
model = Sequential()
model.add(LSTM(256, input_shape=(x.shape[1], x.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


## 跑模型

In [ ]:
model.fit(x, y, nb_epoch=50, batch_size=4096)

Instructions for updating:
Use tf.cast instead.


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """Entry point for launching an IPython kernel.


Epoch 1/50
 24576/276730 [=>............................] - ETA: 8:15 - loss: 4.0088

我们来写个程序，看看我们训练出来的LSTM的效果：

In [ ]:
def predict_next(input_array):
    x = numpy.reshape(input_array, (1, seq_length, 1))
    x = x / float(n_vocab)
    y = model.predict(x)
    return y

def string_to_index(raw_input):
    res = []
    for c in raw_input[(len(raw_input)-seq_length):]:
        res.append(char_to_int[c])
    return res

def y_to_char(y):
    largest_index = y.argmax()
    c = int_to_char[largest_index]
    return c

好，写成一个大程序：

In [ ]:
def generate_article(init, rounds=200):
    in_string = init.lower()
    for i in range(rounds):
        n = y_to_char(predict_next(string_to_index(in_string)))
        in_string += n
    return in_string

In [ ]:
init = 'His object in coming to New York was to engage officers for that service. He came at an opportune moment'
article = generate_article(init)
print(article)